## Imports

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('..')

## DQN Algo Outline

In [4]:
from collections import namedtuple

import torch
from torch import nn
from torch import optim
import numpy as np

import gym
from src import agent, train, utils

## Explore

In [5]:
import random
import time

In [ ]:
rm = utils.ReplayMemory(10000, 100)

In [ ]:
env = gym.envs.make('Pong-v4')

s_t = env.reset()
s_t1 = None

for i in range(20):
    a = random.randrange(env.action_space.n)
    s_t1, r_t, done, _ = env.step(a)
    rm.store(s_t, a, r_t, done)
    
    
    if done:
        s_t = env.reset()

In [ ]:
tr_sample = rm.sample(5)

In [ ]:
tr_sample[0][0].shape

In [21]:
replay_mem_size = int(5e5)
mini_batch_size = 64
num_episodes = int(20)

In [ ]:
agt = agent.DQNAgent()
replay_memory = utils.ReplayMemory(replay_mem_size, mini_batch_size)
obs_history = utils.ObsHistory()
optimizer = optim.RMSprop(agt.qnet.parameters())

env = gym.envs.make('Pong-v4')

for episode in range(num_episodes):  # loop over episodes
    s_t = env.reset()  # reset environment to start new episode
    obs_history.reset(s_t)  # reset observations for new episode
    done = False
        
    cumulative_loss = 0
    n_steps = 0
    ep_r = 0
    while not done:  # loop over steps in episode
        phi_t = obs_history.get_phi()
        a_t = agt.act(phi_t)
        s_t1, r_t, done, _ = env.step(a_t)
        obs_history.store(s_t1)
        n_steps += 1
        ep_r += r_t
        
        # store transition
        phi_t1 = obs_history.get_phi()
        replay_memory.store(s_t, a_t, r_t, done)
        s_t = s_t1
        
        # perform a mini-batch of stochastic gradient descent
        if replay_memory.size() > replay_memory.sample_size + 3:
            mini_batch = replay_memory.sample()
            
            optimizer.zero_grad()
            loss = train.mini_batch_loss(mini_batch, agt)
            loss.backward()
            optimizer.step()
            cumulative_loss += loss.item()
            

    print('Episode #{}. Avg loss: {}. Episode reward: {}'
          .format(episode, cumulative_loss / n_steps, ep_r))
    if episode % 10 == 9:
        torch.save(agt.qnet.state_dict(), 'dqn_agt.pt')

Episode #0. Avg loss: 0.015443761451479193. Episode reward: -21.0
Episode #1. Avg loss: 0.018243928530585273. Episode reward: -20.0


## Manually Go Through One Iteration

In [ ]:
import numpy as np
import torch

import gym
from src import agent, dqn, train, utils

#### Create Gym Environment

In [ ]:
env = gym.envs.make('Pong-v4')

In [ ]:
init_obs = env.reset()

for i in range(20000):
    a = random.randrange(env.action_space.n)
    _, _, done, _ = env.step(a)
    time.sleep(.01)
    env.render()
    
    if done:
        env.reset()

In [ ]:
init_obs = env.reset()
init_obs.shape

In [ ]:
s1, r, done, _ = env.step(0)

In [ ]:
s1.shape

#### Initialize DQN objects

In [ ]:
replay_mem_size = int(1e6)
mini_batch_size = 32

In [ ]:
agt = agent.DQNAgent()
replay_memory = utils.ReplayMemory(replay_mem_size, mini_batch_size)
obs_history = utils.ObsHistory()

#### Begin new episode

In [ ]:
obs_init = env.reset()  # reset environment to start new episode
obs_history.reset(obs_init)  # reset observations for new episode
done = False

#### Choose action

In [ ]:
phi = obs_history.phi
a = agt.act(phi)
obs, rew, done, _ = env.step(a)
obs_history.store(obs)

#### Store Transition

In [ ]:
phi_1 = obs_history.phi
replay_memory.store((phi, a, rew, phi_1, done))

#### Gradient Step

[x] Dummy transitions function for testing.

[x] Make `r` reward vector from transitions.

[x] Make $\max_{a}Q(s', a')$ vector from transitions.

[x] Make `y` target vector from transitions.

In [ ]:
from collections import namedtuple
import torch
from torch import nn
import numpy as np

Transition = namedtuple('Transition', 
                        ['phi', 'a', 'r', 'phi_1', 'done'])

In [ ]:
def dummy_transitions(n):
    transitions = []
    
    for i in range(n):
        phi = torch.empty(4, 84, 84).random_(0, 255)
        phi_1 = torch.empty(4, 84, 84).random_(0, 255)
        a = np.random.randint(0, 6)
        r = np.random.randint(0, 2)
        done = False if np.random.randint(0, 2) == 0 else True
        
        transitions.append(Transition(phi, a, r, phi_1, done))
    
    return transitions

In [ ]:
transitions = dummy_transitions(3)
    
phi, a, r, phi_1, done = zip(*transitions)

In [ ]:
def make_y(transitions, agt):
    y = []
    
    for tr in transitions:
        if tr.done:
            y.append(tr.r)
        else:
            x = tr.phi.unsqueeze(0)
            y.append(tr.r + .99 * agt.get_best_values(x).item())
            
    return torch.Tensor(y)

In [ ]:
y = make_y(transitions, agt)

In [ ]:
def get_max_vals(transitions, agt):
    phis = []

    for tr in transitions:
        phis.append(tr.phi)

    x = torch.stack(phis)
    return agt.get_best_values(x)

In [ ]:
qmax = get_max_vals(transitions, agt)

In [ ]:
loss = nn.MSELoss(reduction='mean')

In [ ]:
loss(y, qmax)

In [ ]:
def mini_batch_loss(transitions, agt):
    y = make_y(transitions, agt)
    qmax = get_max_vals(transitions, agt)
    
    loss = nn.MSELoss(reduction='mean')
    return loss(y, qmax)

In [ ]:
mb_loss = mini_batch_loss(transitions, agt)

#### Optimizer and gradient step

In [ ]:
from torch import optim

optimizer = optim.RMSprop(agt.qnet.parameters())

In [ ]:
optimizer.zero_grad()

In [ ]:
mb_loss = mini_batch_loss(transitions, agt)
mb_loss

In [ ]:
mb_loss.backward()

In [ ]:
optimizer.step()

### Test random agent on Pong

In [ ]:
s0 = env.reset()

In [ ]:
obss = []
obss.append(env.reset())

for _ in range(1000):
    a = np.random.choice(env.action_space.n)
    obs, rew, done, _ = env.step(a)
    obss.append(obs)
    
    if done:
        obs = env.reset()

In [ ]:
env.action_space.n